<a href="https://colab.research.google.com/github/kimhwijin/TensorflowWithKeras/blob/master/Tensorflow2.x/Tensorflow2.x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import timeit
def linear_layer(x):
  return 3 * x + 2

@tf.function
def simple_nn(x):
  return tf.nn.relu(linear_layer(x))

def simple_function(x):
  return 3 * x

#simple_nn #<tensorflow.python.eager.def_function.Function at 0x7f9736390ad0>
#simple_function #<function __main__.simple_function>

cell = tf.keras.layers.LSTMCell(100)

@tf.function
def fn(input, state):
  return cell(input,state)

input = tf.zeros([100,100])
state = [tf.zeros([100,100])] * 2

#워밍업
cell(input,state)
fn(input,state)

graph_time = timeit.timeit(lambda: cell(input,state), number=100)
auto_graph_time = timeit.timeit(lambda: fn(input,state), number=100)
print('graph time :', graph_time)
print('auto graph time :', auto_graph_time)

graph time : 0.13674109100000464
auto graph time : 0.1100486389999844


In [3]:
#functional model

def build_model():
  #가변길이 정수 시퀀스
  text_input_a = tf.keras.Input(shape=(None,), dtype='int32')
  text_input_b = tf.keras.Input(shape=(None,), dtype='int32')

  #1000개의 고유 단어를 128차원 벡터에 매핑하여 임베딩
  shared_embedding = tf.keras.layers.Embedding(1000,128)

  #양쪽 입력을 인코딩하고자 동일한 계층 재사용
  encoded_input_a = shared_embedding(text_input_a)
  encoded_input_b = shared_embedding(text_input_b)

  #최종적으로 2개의 logistic prediction
  prediction_a = tf.keras.layers.Dense(1, activation='sigmoid', name='prediction_a')(encoded_input_a)
  prediction_b = tf.keras.layers.Dense(1, activation='sigmoid', name='prediction_b')(encoded_input_b)

  #가운데 공유모델
  model = tf.keras.Model(inputs=[text_input_a,text_input_b],outputs=[prediction_a,prediction_b])
  
build_model()

In [7]:
#model subclassing 
from tensorflow.keras import layers
class MyLayer(layers.Layer):
  def __init__(self,output_dim,**kwargs):
    self.output_dim = output_dim
    super(MyLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    #계층의 훈련 가능한 가중치 변수 생성
    self.kernel = self.add_weight(name='kernel', shape=(input_shape[1], self.output_dim), initializer='uniform', trainable=True)
  
  def call(self, inputs):
    #예제 입력과 가중치의 곱셈 수행과 반환
    return tf.matmul(inputs,self.kernel)

#output dim 이 20인 mylayer 계층 생성
model = tf.keras.Sequential([MyLayer(20),layers.Activation('softmax')])
                            


In [8]:
#datasets 으로 훈련
import tensorflow_datasets as tfds

#등록된 데이터셋 나열
builders = tfds.list_builders()
print(builders)

#데이터셋의 메타데이터 로드
data, info = tfds.load('mnist', with_info=True)
print(info)

['abstract_reasoning', 'accentdb', 'aeslc', 'aflw2k3d', 'ag_news_subset', 'ai2_arc', 'ai2_arc_with_ir', 'amazon_us_reviews', 'anli', 'arc', 'bair_robot_pushing_small', 'bccd', 'beans', 'big_patent', 'bigearthnet', 'billsum', 'binarized_mnist', 'binary_alpha_digits', 'blimp', 'bool_q', 'c4', 'caltech101', 'caltech_birds2010', 'caltech_birds2011', 'cars196', 'cassava', 'cats_vs_dogs', 'celeb_a', 'celeb_a_hq', 'cfq', 'chexpert', 'cifar10', 'cifar100', 'cifar10_1', 'cifar10_corrupted', 'citrus_leaves', 'cityscapes', 'civil_comments', 'clevr', 'clic', 'clinc_oos', 'cmaterdb', 'cnn_dailymail', 'coco', 'coco_captions', 'coil100', 'colorectal_histology', 'colorectal_histology_large', 'common_voice', 'coqa', 'cos_e', 'cosmos_qa', 'covid19sum', 'crema_d', 'curated_breast_imaging_ddsm', 'cycle_gan', 'deep_weeds', 'definite_pronoun_resolution', 'dementiabank', 'diabetic_retinopathy_detection', 'div2k', 'dmlab', 'downsampled_imagenet', 'dsprites', 'dtd', 'duke_ultrasound', 'emnist', 'eraser_multi_r

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.1,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)



In [12]:
#use datasets with numpy array
import numpy as np
num_items = 100
num_list = np.arange(num_items)

#numpy 배열로 데이터셋 생성
num_list_dataset = tf.data.Dataset.from_tensor_slices(num_list)
print(num_list_dataset)

<TensorSliceDataset shapes: (), types: tf.int64>


In [14]:
datasets, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_dataset = datasets['train']
#batch : batch size = 5
#shuffle : buffer size = 50 , 섞고나서 앞에 50개를 가져옴
#take : take 2 앞에 2개만 가져옴
train_dataset = train_dataset.batch(5).shuffle(50).take(2)
for data in train_dataset:
  print(data)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteYIVBXA/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteYIVBXA/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteYIVBXA/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
(<tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'This film is predictable; it is more predictable then a Vinnie Testaverdi pass, when he huts the ball for the Jets. One saw the ending coming up halfway through the film. The politics reminds me when I was back east. Many people know when the fix is in. I gave this four because of the acting, but the story is lame.',
       b'This was in short a terrible disappointment. By far the worst adaptation of one of my favourite novels. The dialogue was horribly clumsy; I could sense no feeling behind the words expressed by the characters. The lines were delivered too hastily and felt rather out of place. They could just as well have been cited by statues. The chemistry between George C. Scott and Susannah York was non-existent and watching an American Rochester felt strange. He could have at least 

In [16]:
#estimators
BUFFER_SIZE = 10000
BATCH_SIZE = 64

def input_fn(mode):
  datasets , info = tfds.load(name='mnist', with_info=True, as_supervised=True)
  mnist_dataset = (datasets['train'] if mode == tf.estimator.ModeKeys.TRAIN else datasets['test'])

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label
    
  return mnist_dataset.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

test = input_fn('test')
train = input_fn(tf.estimator.ModeKeys.TRAIN)
print(test)
print(train)

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>
<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>
